# CS 5665 - Introduction to Data Science

## Preprocessing

The data used in this project is from [this](https://www.kaggle.com/c/linking-writing-processes-to-writing-quality). 

### Introducing the dataset and packages

In [2]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [6]:
# Import dataset
train_df = pd.read_csv('../data/train_logs.csv')
train_scores = pd.read_csv('../data/train_scores.csv')
test_logs = pd.read_csv('../data/test_logs.csv')

### Cleaning Data and Feature Engineering
Before we can split, we need to convert the keystroke measurements to the appropriate metrics for each user. Due to these metric types originally being event dependent, we will be dropping 'cursor_position', 'down_event', 'up_event', and 'text_change.' 

In [12]:
cleaned_data = train_df.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby('id').sum() 
mins_data = train_df.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby("id").min()
maxes_data = train_df.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby("id").max()
cleaned_data['min_down_time'] = mins_data['down_time']
cleaned_data['max_up_time'] = maxes_data['up_time']
cleaned_data['min_action_time'] = mins_data['action_time']
cleaned_data['max_action_time'] = maxes_data['action_time']
cleaned_data = cleaned_data.merge(train_scores, on='id')

cleaned_test = test_logs.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby('id').sum()
mins_test = test_logs.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby("id").min() 
maxes_test = test_logs.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby("id").max()
cleaned_test['min_down_time'] = mins_test['down_time']
cleaned_test['max_up_time'] = maxes_test['up_time']
cleaned_test['min_action_time'] = mins_test['action_time']
cleaned_test['max_action_time'] = maxes_test['action_time']
cleaned_test = cleaned_test.merge(train_scores, on='id')

cleaned_data.describe()

C:\Users\Nate\AppData\Local\Temp\ipykernel_14700\2082223796.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cleaned_data = train_df.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby('id').sum()
C:\Users\Nate\AppData\Local\Temp\ipykernel_14700\2082223796.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cleaned_test = test_logs.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby('id').sum()


,event_id,down_time,up_time,action_time,word_count,min_down_time,max_up_time,min_action_time,max_action_time,score
count,2.471000e+03,2.471000e+03,2.471000e+03,2.471000e+03,2.471000e+03,2.471000e+03,2.471000e+03,2471.000000,2471.000000,2471.000000
mean,7.033772e+06,2.699550e+09,2.699883e+09,3.336675e+05,7.874151e+05,4.239387e+04,1.769537e+06,0.116957,2989.163497,3.711251
std,7.095160e+06,1.727236e+09,1.727350e+09,1.575202e+05,7.298654e+05,1.658284e+05,2.760283e+05,1.289195,13084.145614,1.024937
min,3.445300e+04,1.723896e+07,1.725651e+07,1.345200e+04,5.044000e+03,1.060000e+02,2.295480e+05,0.000000,153.000000,0.500000
25%,2.406818e+06,1.552195e+09,1.552490e+09,2.111480e+05,2.945275e+05,2.131500e+03,1.754901e+06,0.000000,515.000000,3.000000
50%,4.750903e+06,2.315883e+09,2.316161e+09,3.049510e+05,5.629010e+05,2.114900e+04,1.792315e+06,0.000000,1278.000000,4.000000
75%,9.251451e+06,3.441110e+09,3.441537e+09,4.248140e+05,1.018560e+06,4.592500e+04,1.807774e+06,0.000000,2648.500000,4.500000
max,8.290213e+07,2.424430e+10,2.424494e+10,1.210508e+06,6.546254e+06,7.452424e+06,8.313707e+06,32.000000,447470.000000,6.000000


In order to make the model as simple and linear as possible, each essay's event IDs, down/up times, action times, word counts, and scores have been made into one table by combining the metrics by essay ID (some by max, min, or sum, which is functionally the same as mean in these cases). This will result in massive losses in precision capability but should still produce reasonably good results while being massively faster to work with.

### Preprocessing



In [5]:
# Identify numeric feature statistics
cleaned_data.info()
cleaned_data.describe(include='all')

# Checking for missing values
cleaned_data.describe(include="all").loc['count']
cleaned_data["action_time"].describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2471 entries, 0 to 2470
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2471 non-null   object 
 1   event_id     2471 non-null   int64  
 2   down_time    2471 non-null   int64  
 3   up_time      2471 non-null   int64  
 4   action_time  2471 non-null   int64  
 5   word_count   2471 non-null   int64  
 6   score        2471 non-null   float64
dtypes: float64(1), int64(5), object(1)
memory usage: 154.4+ KB


count    2.471000e+03
mean     3.336675e+05
std      1.575202e+05
min      1.345200e+04
25%      2.111480e+05
50%      3.049510e+05
75%      4.248140e+05
max      1.210508e+06
Name: action_time, dtype: float64

In [45]:
passthrough_variables = ['id']
scale_variables = ['event_id', 'down_time', 'up_time', 'action_time', 'word_count', 'score', 
                   'min_down_time', 'max_up_time', 'min_action_time', 'max_action_time']

In [46]:
preprocessor = make_column_transformer(
    ("passthrough", passthrough_variables),
    (StandardScaler(), scale_variables)
)

In [47]:
transformed = preprocessor.fit_transform(cleaned_data)
column_names = passthrough_variables + scale_variables

In [48]:
X_trained_transformed = pd.DataFrame(transformed, columns=column_names)
X_trained_transformed

,id,event_id,down_time,up_time,action_time,word_count,score
0,001519c8,-0.530521,-0.307346,-0.307347,-0.231284,-0.630137,-0.206152
1,0022f953,-0.566908,-0.825925,-0.825904,-0.370037,-0.464609,-0.206152
2,0042269b,0.214492,0.421043,0.421065,0.55581,0.024893,2.233515
3,0059420b,-0.820785,-0.855492,-0.855519,-0.914808,-0.858118,-1.669952
4,0075873a,-0.539847,-0.517724,-0.517701,-0.126774,-0.645222,0.281781
...,...,...,...,...,...,...,...
2466,ffb8c745,0.591743,0.456797,0.456863,1.054063,0.585768,-0.206152
2467,ffbef7e5,-0.51342,-0.293632,-0.293682,-0.758447,-0.283245,0.281781
2468,ffccd6fd,-0.330046,0.616673,0.616573,-0.648223,-0.417585,-2.157886
2469,ffec5b38,-0.250485,-0.48091,-0.480904,-0.280837,-0.164217,1.257648


### Splitting Data

In [49]:
# for run time consistency, set seed
np.random.seed(42)
train_df, test_df = train_test_split(X_trained_transformed, test_size=0.2, random_state=42)


### Dummy Submission

In [50]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(train_df.drop(columns=['score']), train_df['score'])

DummyClassifier(strategy='most_frequent')

In [51]:
# Predict
prediction = pd.DataFrame()
prediction['id'] = cleaned_test.index
prediction ['score']= dummy_clf.predict(cleaned_test)
print(prediction.info())

prediction.to_csv('../data/prediction.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      3 non-null      object 
 1   score   3 non-null      float64
dtypes: float64(1), object(1)
memory usage: 180.0+ bytes
None


## Dummy Submission Checkpoint 2 (1)

In [52]:
dummy_clf_2_1 = DummyClassifier(strategy="uniform")
dummy_clf_2_1.fit(train_df.drop(columns=['score']), train_df['score'])

DummyClassifier(strategy='uniform')

In [53]:
# Predict
prediction = pd.DataFrame()
prediction['id'] = cleaned_test.index
prediction ['score']= dummy_clf.predict(cleaned_test)

prediction.to_csv('../data/prediction_2_1.csv', index=False)

prediction

,id,score
0,0000aaaa,0.281781
1,2222bbbb,0.281781
2,4444cccc,0.281781


## Dummy Submission Checkpoint 2 (2)

In [54]:
dummy_clf_2_2 = DummyClassifier(strategy="prior")
dummy_clf_2_2.fit(train_df.drop(columns=['score']), train_df['score'])

DummyClassifier()

In [55]:
# Predict
prediction = pd.DataFrame()
prediction['id'] = cleaned_test.index
prediction ['score']= dummy_clf.predict(cleaned_test)

prediction.to_csv('../data/prediction_2_2.csv', index=False)

prediction

,id,score
0,0000aaaa,0.281781
1,2222bbbb,0.281781
2,4444cccc,0.281781


# Model Selection
We will compare the default efficiency of each of the following regression models:
- Ordinary Least Squares
- Ridge Regression
- Support Vector
- Decision Tree
- Random Forest